<u><h1 style="color:#21B6A8"> Exercice : Dataset cleaning and mapinulation </h1></u>

<h2 style="color:#98D7C2">Importing the Pandas library 🐼</h2>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<h2 style="color:#98D7C2">Reading datasets</h2>

In [ ]:
customers = pd.read_csv('dataset/customers.csv')
products = pd.read_csv('dataset/products.csv')
transactions = pd.read_csv('dataset/transactions.csv')

<h2 style="color:#98D7C2">Displaying data </h2>

In [ ]:
#customers.head()
#products.head(20) 
#transactions.head()

<h2 style="color:#98D7C2">Data description</h2>

In [ ]:
transactions.describe()

In [ ]:
transactions[~transactions['date'].astype(str).str.contains("test", case=False, na=False)]

<i style="color:#90ADC6"> -> ici, je vois que la valeur "test" revient trop souvent dans la colonne date, ce n’est pas une date valide, pandas considère toute la colonne comme un objet. Après avoir analysé les lignes avec cette valeur, je décide de supprimer ces lignes car ca ressemble à des valeurs de test. </i>

<h2 style="color:#98D7C2">Deleting ‘test’ values</h2>

In [ ]:
transactions = transactions[~transactions['date'].astype(str).str.contains("test", case=False, na=False)]

<h2 style="color:#98D7C2">Checking data types</h2>

In [ ]:
transactions.dtypes

<i style="color:#90ADC6"> -> ici, je m'aperçois que la date est au format objet, ce n'est pas correct, je vais donc la convertir en datetime. </i>

<h2 style="color:#98D7C2">Converting date to date</h2>

In [ ]:
transactions['date'] = pd.to_datetime(transactions['date'])

<h2 style="color:#98D7C2">Checking for duplicates</h2>

In [ ]:
transactions.duplicated().sum()
products.duplicated().sum()
customers.duplicated().sum()

<h2 style="color:#98D7C2">Checking for null values</h2>

In [ ]:
transactions.isnull().sum()
products.isnull().sum()
customers.isnull().sum()

<h2 style="color:#98D7C2">Correction negative values</h2>

In [ ]:
products.describe()

In [ ]:
products['price'].min()

In [ ]:
products = products[products['price']>=0]

<h2 style="color:#98D7C2">Display monthly revenue trend</h2>

In [ ]:
merged = pd.merge(transactions, products[['price', 'id_prod']], on='id_prod')
merged['month'] = merged['date'].dt.to_period('M')
ca_mois = merged.groupby('month')['price'].sum().reset_index()
ca_mois

<i style="color:#90ADC6"> -> Ici, je choisis de faire une jointure interne car je souhaite uniquement conserver les transactions pour lesquelles le produit existe dans la table des produits. </i>

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(ca_mois['month'].astype(str), ca_mois['price'], marker='x', linestyle='-')
plt.title("Chiffre d'affaires par mois")
plt.xlabel("Mois")
plt.ylabel("Chiffre d'affaires (€)")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

<h2 style="color:#98D7C2">Calculates the number of transactions per customer</h2>

In [ ]:
transactions_client = transactions.groupby('client_id')['id_prod'].count().reset_index(name='nmb_of_transactions')
transactions_client

In [ ]:
transactions_client['client_id'].isnull().sum()

In [ ]:
transactions_client[transactions_client['nmb_of_transactions'] == 0]

In [90]:
transactions_client.duplicated().sum()

0

In [91]:
transactions_client['nmb_of_transactions'].describe()

count     8600.000000
mean        39.164651
std        156.448296
min          1.000000
25%         14.000000
50%         25.000000
75%         47.000000
max      12855.000000
Name: nmb_of_transactions, dtype: float64

In [97]:
transactions_client[transactions_client['nmb_of_transactions'] > 1000]

,client_id,nmb_of_transactions
677,c_1609,12855
2724,c_3454,3275
4388,c_4958,2562
6337,c_6714,4473


In [108]:
# Vérifier l'existence dans customers
clients_suspects = transactions_client[transactions_client['nmb_of_transactions'] > 1000]['client_id']
customers[customers['client_id'].isin(clients_suspects)]

,client_id,sex,birth
1378,c_4958,m,1999
1911,c_6714,f,1968
3642,c_1609,m,1980
8088,c_3454,m,1969
